In [1]:
%%capture
!pip install tables
!pip install keras-tuner
import os, gc, pickle, datetime, scipy.sparse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style

from sklearn.model_selection import GroupKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display

import scipy.sparse
import scprep

VERBOSE = 0

In [11]:
def correlation_score(y_true, y_pred):
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(y_true.shape[0]):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

def negative_correlation_loss(y_true, y_pred):
    my = K.mean(tf.convert_to_tensor(y_pred), axis=1)
    my = tf.tile(tf.expand_dims(my, axis=1), (1, tf.shape(y_true)[1]))
    ym = y_pred - my
    r_num = K.sum(tf.multiply(y_true, ym), axis=1)
    r_den = tf.sqrt(K.sum(K.square(ym), axis=1) * float(tf.shape(y_true)[-1]))
    r = tf.reduce_mean(r_num / r_den)
    return - r

In [148]:
%%capture
!pip install keras_tuner
import keras_tuner

In [152]:
from sklearn.model_selection import train_test_split

In [174]:
targets.shape

(10594, 22858)

In [177]:
metadata_df = pd.read_csv('../../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta = metadata_df.reindex(cell_index)
meta = meta.iloc[:10592]

In [179]:
meta.groupby('day').count()

,donor,cell_type,technology
day,,,
2,7264,7264,7264
3,3328,3328,3328


In [180]:
metadata_df = pd.read_csv('../../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta = metadata_df.reindex(cell_index)

In [182]:
meta.loc[meta.day == 7]

,day,donor,cell_type,technology
cell_id,,,,
611fc1195807,7,32606,MasP,multiome
d8ac427860ab,7,32606,EryP,multiome
a5d9dab35d0d,7,32606,HSC,multiome
17f3beafca35,7,32606,NeuP,multiome
44ebe48354cf,7,32606,NeuP,multiome
...,...,...,...,...
063cead1a4ea,7,31800,HSC,multiome
553bca99ba78,7,31800,MasP,multiome
00783f28b463,7,31800,NeuP,multiome


In [176]:
Y = targets[:-2]
tuner = keras_tuner.BayesianOptimization(
    create_model,
    overwrite=True,
    objective=keras_tuner.Objective("val_negative_correlation_loss", direction="min"),
    max_trials=30,
    directory='/kaggle/temp',
    seed=1)
lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                       patience=6, verbose=0)
es = EarlyStopping(monitor="val_loss",
                   patience=3, 
                   verbose=0,
                   mode="min", 
                   restore_best_weights=True)
callbacks = [lr, es, tf.keras.callbacks.TerminateOnNaN()]
X_tr, X_va, y_tr, y_va = train_test_split(X,Y, test_size=0.2, random_state=10)
tuner.search(X_tr, y_tr,
             epochs=1000,
             validation_data=(X_va, y_va),
             batch_size=BATCH_SIZE,
             callbacks=callbacks, verbose=2)
del X_tr, X_va, y_tr, y_va, lr, es, callbacks

tuner.results_summary()
# Table of the 10 best trials
display(pd.DataFrame([hp.values for hp in tuner.get_best_hyperparameters(10)]))

# Keep the best hyperparameters
best_hp = tuner.get_best_hyperparameters(1)[0]

Trial 30 Complete [00h 02m 28s]
val_negative_correlation_loss: -0.674669623374939

Best val_negative_correlation_loss So Far: -0.6767739057540894
Total elapsed time: 01h 06m 55s
Results summary
Results in /kaggle/temp/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
reg1: 0.0001
reg2: 3.307001554236342e-07
units1: 256
units2: 256
units3: 256
lr: 0.001
Score: -0.6767739057540894
Trial summary
Hyperparameters:
reg1: 0.0001
reg2: 8.4533922711223e-08
units1: 256
units2: 1024
units3: 1024
lr: 0.001
Score: -0.6767122149467468
Trial summary
Hyperparameters:
reg1: 0.0001
reg2: 1e-08
units1: 256
units2: 256
units3: 1024
lr: 0.001
Score: -0.6766196489334106
Trial summary
Hyperparameters:
reg1: 2.621834283020877e-06
reg2: 3.5043930455315717e-07
units1: 512
units2: 256
units3: 1024
lr: 0.001
Score: -0.6765390634536743
Trial summary
Hyperparameters:
reg1: 3.4501592621734256e-06
reg2: 2.693819104730028e-07
units1: 1024
units2: 256
units3: 512
lr: 0.001
Score: -0.676396608352661

,reg1,reg2,units1,units2,units3,lr
0,1.000000e-04,3.307002e-07,256,256,256,0.001
1,1.000000e-04,8.453392e-08,256,1024,1024,0.001
2,1.000000e-04,1.000000e-08,256,256,1024,0.001
3,2.621834e-06,3.504393e-07,512,256,1024,0.001
4,3.450159e-06,2.693819e-07,1024,256,512,0.001
5,1.268217e-05,1.919784e-07,512,512,512,0.001
6,1.332574e-06,1.221609e-06,256,256,1024,0.001
7,3.978453e-07,1.000000e-08,1024,256,1024,0.001
8,1.000000e-04,3.084518e-07,1024,256,1024,0.001
9,1.000000e-04,1.000000e-08,256,1024,256,0.001


In [175]:
# %%capture
# !pip install --upgrade --force-reinstall tensorflow
# import math
# import tensorflow as tf
# import tensorflow.keras.backend as K
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
# from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, BatchNormalization
# LR_START = 0.01
# BATCH_SIZE = 512

best_hp = keras_tuner.HyperParameters()
best_hp.values = {'reg1': 8e-5,
                'reg2': 2e-5,
                'units1': 256,
                'units2': 512,
                'units3': 512,
                'lr': 0.01
                 }

# best_hp = keras_tuner.HyperParameters()
# best_hp.values = {'reg1': 0.0001,
#                 'reg2': 1e-08,
#                 'units1': 256,
#                 'units2': 1024,
#                 'units3': 1024,
#                 'lr': 0.001
#                  }

def create_model(hp):
    
    #reg1 = 8e-5
    #reg2 = 2e-5
    # reg1 = 9.613e-06
    # reg2 = 1e-07
    reg1 = hp.Float("reg1", min_value=1e-8, max_value=1e-4, sampling="log")
    reg2 = hp.Float("reg2", min_value=1e-8, max_value=1e-4, sampling="log")
    
    REG1 = tf.keras.regularizers.l2(reg1)
    REG2 = tf.keras.regularizers.l2(reg2)
    
    DROP = 0.1

    activation = 'selu'
    inputs = Input(shape =(X.shape[1],))

    x0 = Dense(hp.Choice('units1', [1024,512,256]), 
              kernel_regularizer = REG1,
              activation = activation,
             )(inputs)
    x0 = Dropout(DROP)(x0)
    
    x1 = Dense(hp.Choice('units2', [1024,512,256]), 
               kernel_regularizer = REG1,
               activation = activation,
             )(x0)
    x1 = Dropout(DROP)(x1)
    
    x2 = Dense(hp.Choice('units3', [1024,512,256]), 
               kernel_regularizer = REG1,
               activation = activation,
             )(x1) 
    x2= Dropout(DROP)(x2)

    x = Concatenate()([
                x0, 
                x1, 
                x2
                ])
    x = Dense(Y.shape[1], 
                kernel_regularizer = REG2,
                activation='linear',
                )(x)
    model = Model(inputs, x)
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp.Float("lr", min_value=1e-3, max_value=1e-0)),
        metrics = [negative_correlation_loss],
        loss = negative_correlation_loss
         )
    return model

In [58]:
inputs_both_128 = np.genfromtxt("../multi_pca/inputs_both_128.csv", delimiter=',')
multi_X_train = inputs_both_128[:105942]
targets = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
import scprep
targets = scprep.filter.remove_empty_genes(targets)
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
targets = scprep.filter.remove_empty_genes(targets)
targets -= targets.mean(axis=1).reshape(-1, 1)
targets /= targets.std(axis=1).reshape(-1, 1)

pca_train = TruncatedSVD(n_components=512, random_state=1)
pca_targets = pca_train.fit_transform(targets)

with np.load('../sparse/train_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

with np.load('../sparse/test_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index_t = data['index']

metadata_df = pd.read_csv('../../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta = metadata_df.reindex(cell_index)

X = multi_X_train
Y = pca_targets

In [40]:
Y_true = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
Y_true = scprep.filter.remove_empty_genes(Y_true)
Y_true = Y_true[:int(Y_true.shape[0]/100)]
Y_true -= Y_true.mean(axis=1).reshape(-1, 1)
Y_true /= Y_true.std(axis=1).reshape(-1, 1)
svd = TruncatedSVD(n_components = 128,random_state = 1)
Y = svd.fit_transform(Y_true)
Y_true = np.array(Y_true)

X = np.genfromtxt('../multi_pca/tenth/X.csv',delimiter=',')
# X -= X.mean(axis=1).reshape(-1, 1)
# X /= X.std(axis=1).reshape(-1, 1)
X = X[:,:128]

with np.load('../sparse/train_multi_inputs_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']

metadata_df = pd.read_csv('../../input/open-problems-multimodal/metadata.csv', index_col='cell_id')
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta = metadata_df.reindex(cell_index)
meta = meta[:int(len(meta)/100)]

In [45]:
svd.components_.shape

(128, 22858)

In [59]:
X = X[:int(len(X)/10)]
Y = Y[:int(len(Y)/10)]
meta = meta[:int(len(meta)/10)]
targets = targets[:int(len(targets)/10)]
%mkdir tenth
%cd tenth
np.save('X.npy',X)
np.save('Y.npy',Y)
np.save('meta.npy',meta)
np.save('targets.npy',targets)
np.save('components.npy',pca_train.components_)
%cd ..

/home/jovyan/kaggle/working/all_in_one/tenth
/home/jovyan/kaggle/working/all_in_one


In [136]:
X_pca = np.load('tenth/X.npy')
Y = np.load('tenth/Y.npy')
targets = np.load('tenth/targets.npy')
components = np.load('tenth/components.npy')

In [137]:
meta.shape

(10594, 4)

In [142]:
X_pca = X_pca[:-2]
X_pca.shape

(10592, 128)

In [37]:
correlation_score(targets[:int(len(targets)/10)][idx_va], pca_train.inverse_transform(y_va_pred))#pred_train@pca_target_comps)

0.6672151897833535

In [53]:
targets.shape

(1059, 22858)

In [159]:
X = np.load("/kaggle/working/gene_annot/train_multi_geneactivity.npy")

In [127]:
X.shape

(10592, 19609)

In [62]:
svd_X = TruncatedSVD(n_components = 512,random_state = 1)
X = svd_X.fit_transform(X)[:,:128]

In [96]:
to_keep = np.load('/kaggle/working/gene_annot/to_keep.npy')

In [128]:
X = X[:,to_keep]

In [160]:
X = np.hstack([X,X_pca])

In [134]:
X.shape

(10592, 11130)

In [99]:
X.shape

(10592, 11002)

In [103]:
targets.shape

(105942, 23418)

In [95]:
to_keep_y = np.load('/kaggle/working/gene_annot/to_keep_y.npy')

In [125]:
with np.load('../../working/sparse/train_multi_targets_idxcol.npz',allow_pickle=True) as data:
    cell_index = data['index']
    cell_columns = data['columns']
cell_columns[21664]

'ENSG00000000003'

In [102]:
targets = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")

In [84]:
targets = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
Y = targets[:int(targets.shape[0]/10),to_keep_y].toarray()

In [104]:
Y = targets[:int(targets.shape[0]/10),to_keep_y].toarray()

In [124]:
to_keep_y

array([21664,  9634, 18887, ...,  7146,  1056,  4660])

In [123]:
to_keep

array([19091, 17428,  1042, ...,  9031, 16476, 10740])

In [115]:
10592/4

2648.0

In [109]:
X.shape

(10592, 11002)

In [114]:
correlation_score(X, Y)

0.07904437266109401

In [113]:
X[:,2].mean()

0.48623547

In [112]:
Y[:,2].mean()

0.84786725

In [90]:
X.shape

(10592, 11002)

In [91]:
Y.shape

(10594, 11002)

In [172]:
import warnings
warnings.filterwarnings("ignore")
VERBOSE = 2

pred_train = np.zeros_like(Y)

N_SPLIT = 3
#kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
kf = KFold(n_splits = N_SPLIT)

score_list = []

for fold,(idx_tr, idx_va) in enumerate(kf.split(X)):
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    
    X_va = X[idx_va]
    y_va = Y[idx_va]
    
    #day = (meta.iloc[idx_va].iloc[0]).day
    
    #model = my_model(n_inputs=X.shape[1],n_outputs=Y.shape[1],reg1=reg1,reg2=reg2,loss = negative_correlation_loss)
    model = create_model()
    #weights = meta.iloc[idx_tr].apply(lambda x:after_ratio[x.cell_type],axis=1)    
    
    lr = ReduceLROnPlateau(
                monitor = "val_loss",
                factor = 0.9, 
                patience = 4, 
                verbose = VERBOSE)
    
    es = EarlyStopping(
                monitor = "val_loss",
                patience = 30, 
                verbose = VERBOSE,
                mode = "min", 
                restore_best_weights = True)

    model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2),
            metrics = [negative_correlation_loss],
            loss = negative_correlation_loss
             )

    model.fit(X_tr,
              y_tr,
              validation_data=(X_va,y_va),
              epochs =500,
              verbose = VERBOSE,
              batch_size=256,
              callbacks = [es,lr]
             )
    y_va_pred = model.predict(X_va)
    
    print(f'\n --------- FOLD {fold} -----------')
    
    corrscore = correlation_score(targets[idx_va], y_va_pred@components)
    pred_train[idx_va] = y_va_pred
    
    print(f"Fold {fold}, correlation =  {corrscore:.5f}")
   
    model.save(f"./submissions/multi_model_{fold}")

    print('model saved')
        
    del X_tr,X_va,y_tr,y_va
    gc.collect()
    score_list.append(corrscore)
    
# Show overall score
print(f"{Fore.GREEN}{Style.BRIGHT}Mean corr = {np.array(score_list).mean():.5f}{Style.RESET_ALL}")
# score_total = correlation_score(Y_true, pca_train.inverse_transform(pred_train))#pred_train@pca_target_comps)
# print(f"{Fore.BLUE}{Style.BRIGHT}Oof corr   = {score_total:.5f}{Style.RESET_ALL}")

Epoch 1/500
28/28 - 2s - loss: -3.0043e+00 - negative_correlation_loss: -3.7506e+00 - val_loss: -3.5300e+00 - val_negative_correlation_loss: -4.4791e+00 - lr: 0.0100 - 2s/epoch - 57ms/step
Epoch 2/500
28/28 - 1s - loss: -3.5776e+00 - negative_correlation_loss: -4.3859e+00 - val_loss: -3.8600e+00 - val_negative_correlation_loss: -4.5069e+00 - lr: 0.0100 - 603ms/epoch - 22ms/step
Epoch 3/500
28/28 - 1s - loss: -3.8612e+00 - negative_correlation_loss: -4.3968e+00 - val_loss: -4.0710e+00 - val_negative_correlation_loss: -4.5033e+00 - lr: 0.0100 - 614ms/epoch - 22ms/step
Epoch 4/500
28/28 - 1s - loss: -4.0331e+00 - negative_correlation_loss: -4.3989e+00 - val_loss: -4.2031e+00 - val_negative_correlation_loss: -4.5064e+00 - lr: 0.0100 - 604ms/epoch - 22ms/step
Epoch 5/500
28/28 - 1s - loss: -4.1389e+00 - negative_correlation_loss: -4.4003e+00 - val_loss: -4.2834e+00 - val_negative_correlation_loss: -4.5042e+00 - lr: 0.0100 - 594ms/epoch - 21ms/step
Epoch 6/500
28/28 - 1s - loss: -4.2089e+00 

KeyboardInterrupt: 

In [39]:
correlation_score(targets[:int(len(targets)/10)], pca_train.inverse_transform(pred_train))#pred_train@pca_target_comps)

0.6363475576952972

In [31]:
pca_train.inverse_transform(pred_train).shape

(10594, 22858)

In [32]:
targets.shape

(105942, 22858)

In [33]:
correlation_score(targets[:int(len(targets)/10)], pca_train.inverse_transform(pred_train))#pred_train@pca_target_comps)

0.6179052437901781

In [ ]:
print(f"{Fore.BLUE}{Style.BRIGHT}Oof corr   = {score_total:.5f}{Style.RESET_ALL}")

In [23]:
correlation_score(targets[idx_va], pca_train.inverse_transform(y_va_pred))

0.5752172123203347

In [14]:
Y_true.shape

(1059, 22858)

In [185]:
LR_START = 0.01
BATCH_SIZE = 512

def create_model():
    
    reg1 = 9.613e-06
    reg2 = 1e-07
    REG1 = tf.keras.regularizers.l2(reg1)
    REG2 = tf.keras.regularizers.l2(reg2)
    DROP = 0.1

    activation = 'selu'
    inputs = Input(shape =(X.shape[1],))

    x0 = Dense(256, 
              kernel_regularizer = REG1,
              activation = activation,
             )(inputs)
    x0 = Dropout(DROP)(x0)
    
    
    x1 = Dense(512, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x0)
    x1 = Dropout(DROP)(x1)
    
    
    x2 = Dense(512, 
               kernel_regularizer = REG1,
               activation = activation,
             )(x1) 
    x2= Dropout(DROP)(x2)
    
    x3 = Dense(Y.shape[1],
               kernel_regularizer = REG1,
               activation = activation,
             )(x2)
    x3 = Dropout(DROP)(x3)

         
    x = Concatenate()([
                x0, 
                x1, 
                x2, 
                x3
                ])
    
    x = Dense(Y.shape[1], 
                kernel_regularizer = REG2,
                activation='linear',
                )(x)
    
    
    model = Model(inputs, x)
    

    return model

In [186]:
X = X_pca[:,:40]
svd = TruncatedSVD(n_components = 512,random_state = 1)
Y = svd.fit_transform(targets)

In [189]:
Y = Y[:10592]

In [192]:
meta = meta[:10592]

In [198]:
targets = scipy.sparse.load_npz("../sparse/train_multi_targets_values.sparse.npz")
targets = targets
targets = scprep.filter.remove_empty_genes(targets)
# Normalize the targets row-wise: This doesn't change the correlations,
# and negative_correlation_loss depends on it
targets -= targets.mean(axis=1).reshape(-1, 1)
targets /= targets.std(axis=1).reshape(-1, 1)
svd = TruncatedSVD(n_components = 512,random_state = 1)
Y = svd.fit_transform(targets)
inputs_both_128 = np.load("../multi_pca/inputs_both_128.npy")
multi_X_train = inputs_both_128[:len(targets),:40]

In [199]:
X = multi_X_train

In [212]:
X = X[:int(len(X)/10)]
Y = Y[:int(len(Y)/10)]

In [213]:
import warnings
warnings.filterwarnings("ignore")

VERBOSE = 0
N_SPLIT = 3
#kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
kf = KFold(n_splits = 3)

for fold,(idx_tr, idx_va) in enumerate(kf.split(X)):
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    
    X_va = X[idx_va]
    y_va = Y[idx_va] 
    
    model = create_model()
    
    lr = ReduceLROnPlateau(
                monitor = "val_loss",
                factor = 0.9, 
                patience = 4, 
                verbose = VERBOSE)
    
    es = EarlyStopping(
                monitor = "val_loss",
                patience = 30, 
                verbose = VERBOSE,
                mode = "min", 
                restore_best_weights = True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss = 'mse',
                  metrics=None)
    model.fit(X_tr,
              y_tr,
              validation_data=(X_va,y_va),
              epochs =500,
              verbose = VERBOSE,
              batch_size=256,
              callbacks = [es,lr]
             )
    pred = model.predict(X_va)
    
    print(f'\n --------- FOLD {fold} -----------')
    print(f'Mean squared error = {np.round(mean_squared_error(y_va,pred),2)}')
    print(f'Correlation = {np.round(correlation_score(targets[idx_va], pred@svd.components_),2)}')

    filename = f"model_{fold}"
    model.save(filename)
    print('model saved :',filename)
        
    del X_tr,X_va,y_tr,y_va
    gc.collect()

111/111 [==============================] - 1s 6ms/step

 --------- FOLD 0 -----------
Mean squared error = 2.130000114440918
Correlation = 0.69
model saved : model_0
111/111 [==============================] - 1s 6ms/step

 --------- FOLD 1 -----------
Mean squared error = 2.1600000858306885
Correlation = 0.69
model saved : model_1
111/111 [==============================] - 1s 5ms/step

 --------- FOLD 2 -----------
Mean squared error = 3.140000104904175
Correlation = 0.66
model saved : model_2


In [216]:
import warnings
warnings.filterwarnings("ignore")

VERBOSE = 0
N_SPLIT = 3
#kf = KFold(n_splits=N_SPLIT, shuffle=True, random_state=42)
kf = KFold(n_splits = 3)

for fold,(idx_tr, idx_va) in enumerate(kf.split(X)):
    
    X_tr = X[idx_tr]
    y_tr = Y[idx_tr]
    
    X_va = X[idx_va]
    y_va = Y[idx_va] 
    
    model = create_model()
    
    lr = ReduceLROnPlateau(
                monitor = "val_loss",
                factor = 0.9, 
                patience = 4, 
                verbose = VERBOSE)
    
    es = EarlyStopping(
                monitor = "val_loss",
                patience = 30, 
                verbose = VERBOSE,
                mode = "min", 
                restore_best_weights = True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2),
                  loss = negative_correlation_loss,
                  metrics=negative_correlation_loss)
    model.fit(X_tr,
              y_tr,
              validation_data=(X_va,y_va),
              epochs =500,
              verbose = VERBOSE,
              batch_size=256,
              callbacks = [es,lr]
             )
    pred = model.predict(X_va)
    
    print(f'\n --------- FOLD {fold} -----------')
    print(f'Mean squared error = {np.round(mean_squared_error(y_va,pred),2)}')
    print(f'Correlation = {np.round(correlation_score(targets[idx_va], pred@svd.components_),2)}')

    filename = f"model_{fold}"
    model.save(filename)
    print('model saved :',filename)
        
    del X_tr,X_va,y_tr,y_va
    gc.collect()

111/111 [==============================] - 1s 6ms/step

 --------- FOLD 0 -----------
Mean squared error = 26272.58984375
Correlation = 0.41
model saved : model_0
111/111 [==============================] - 1s 5ms/step

 --------- FOLD 1 -----------
Mean squared error = 30454.01953125
Correlation = 0.41
model saved : model_1
111/111 [==============================] - 1s 4ms/step

 --------- FOLD 2 -----------
Mean squared error = 33246.2109375
Correlation = 0.37
model saved : model_2
